In [34]:
df_lxml = pd.read_csv("../data/interim/assemblee_extrait_lxml.csv", encoding='utf-8')

In [35]:
df_base = pd.read_csv("../data/interim/extract_16.csv", low_memory=False, dtype={"ID_orateur": str})

In [37]:
id_paragraphes_lxml = set(df_lxml['ID_paragraphe'].astype(str))
id_paragraphes_base = set(df_base['ID_paragraphe'].astype(str))

print("ID_paragraphes in df_lxml but not in df_base:")
print(id_paragraphes_lxml - id_paragraphes_base)

print("\nID_paragraphes in df_base but not in df_lxml:")
print(id_paragraphes_base - id_paragraphes_lxml)

ID_paragraphes in df_lxml but not in df_base:
{'3418711', '3421665', '3418707', '2830424', '3418710', '3418708', '3463669', '3418709', '3004162', '2899553', '2928894'}

ID_paragraphes in df_base but not in df_lxml:
set()


In [ ]:
ids_to_show = {'3418711', '3421665', '3418707', '2830424', '3418710', '3418708', '3463669', '3418709', '3004162', '2899553', '2928894'}
df_lxml[df_lxml['ID_paragraphe'].astype(str).isin(ids_to_show)]

,UID,SeanceRef,SessionRef,DateSeance,DateSeanceJour,NumSeanceJour,NumSeance,TypeAssemblee,Legislature,Session,...,Ordre_seance,Code_grammaire,Code_style,Code_parole,Role_debat,Nom_orateur,Qualite_orateur,ID_orateur,stime,Texte
95971,CRSANR5L16S2022E1N013,RUANR5L16S2022IDS26211,SCR5A2022E1,20220721090000000,jeudi 21 juillet 2022,1,13,AN,16,Première session extraordinaire 2022,...,678,INTERRUPTION_1_10,NORMAL,NaN,NaN,M. Erwan Balanant,NaN,719558.0,NaN,NaN
108114,CRSANR5L16S2024O1N200,RUANR5L16S2024IDS28364,SCR5A2024O1,20240517213000000,vendredi 17 mai 2024,3,200,AN,16,Session ordinaire 2023-2024,...,879,FIN_SEAN_2_5,Signature droite,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135277,CRSANR5L16S2024O1N159,RUANR5L16S2024IDS28181,SCR5A2024O1,20240326213000000,mardi 26 mars 2024,3,159,AN,16,Session ordinaire 2023-2024,...,605,FIN_SEAN_2_5,Signature droite,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135278,CRSANR5L16S2024O1N159,RUANR5L16S2024IDS28181,SCR5A2024O1,20240326213000000,mardi 26 mars 2024,3,159,AN,16,Session ordinaire 2023-2024,...,606,FIN_SEAN_2_5,Signature droite,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135279,CRSANR5L16S2024O1N159,RUANR5L16S2024IDS28181,SCR5A2024O1,20240326213000000,mardi 26 mars 2024,3,159,AN,16,Session ordinaire 2023-2024,...,607,FIN_SEAN_2_5,Signature droite,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135280,CRSANR5L16S2024O1N159,RUANR5L16S2024IDS28181,SCR5A2024O1,20240326213000000,mardi 26 mars 2024,3,159,AN,16,Session ordinaire 2023-2024,...,608,FIN_SEAN_2_5,Signature droite,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135281,CRSANR5L16S2024O1N159,RUANR5L16S2024IDS28181,SCR5A2024O1,20240326213000000,mardi 26 mars 2024,3,159,AN,16,Session ordinaire 2023-2024,...,609,FIN_SEAN_2_5,Signature droite,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154840,CRSANR5L16S2024O1N164,RUANR5L16S2024IDS28200,SCR5A2024O1,20240402150000000,mardi 02 avril 2024,2,164,AN,16,Session ordinaire 2023-2024,...,1024,FIN_SEAN_1_0,NORMAL,NaN,president,NaN,NaN,NaN,NaN,NaN
232904,CRSANR5L16S2023O1N136,RUANR5L16S2023IDS26782,SCR5A2023O1,20230208150000000,mercredi 08 février 2023,1,136,AN,16,Session ordinaire 2022-2023,...,1174,FIN_SEAN_2_5,Signature droite,NaN,NaN,NaN,NaN,NaN,NaN,NaN
242275,CRSANR5L16S2023O1N040,RUANR5L16S2023IDS26422,SCR5A2023O1,20221028213000000,vendredi 28 octobre 2022,3,40,AN,16,Session ordinaire 2022-2023,...,1213,RETRAIT_ADT,Info Italiques,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
df_base

,UID,SeanceRef,SessionRef,DateSeance,DateSeanceJour,NumSeanceJour,NumSeance,TypeAssemblee,Legislature,Session,...,Code_grammaire,Code_style,Code_parole,Role_debat,Nom_orateur,Qualite_orateur,ID_orateur,stime,Texte,Source_extraction
0,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,OUV_SEAN_2_1,NORMAL,NaN,president,Mme la présidente,NaN,719874,986.96,La séance est ouverte.,global
1,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,OUV_SEAN_2_2,Info Italiques,NaN,NaN,NaN,NaN,NaN,NaN,(La séance est ouverte à seize heures.),global
2,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,ODJ_APPEL_DISCUSSION,NORMAL,NaN,president,Mme la présidente,NaN,719874,990.36,L’ordre du jour appelle la discussion du proje...,point
3,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,PAROLE_GENERIQUE,NORMAL,PAROLE_1_1,NaN,Mme la présidente,NaN,719874,1007.69,La parole est à Mme la ministre de la transiti...,point
4,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,PAROLE_GENERIQUE,NORMAL,PAROLE_1_2,NaN,Mme Agnès Pannier-Runacher,ministre de la transition énergétique,759832,1041.01,"Après la promulgation, vendredi, de la loi rel...",point
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337025,CRSANR5L16S2023O1N156,RUANR5L16S2023IDS26837,SCR5A2023O1,20230227160000000,lundi 27 février 2023,1,156,AN,16,Session ordinaire 2022-2023,...,PAROLE_GENERIQUE,NORMAL,PAROLE_1_2,NaN,M. Stanislas Guerini,ministre,721498,16561.60,Je le dis sous le contrôle de Mme la président...,point
337026,CRSANR5L16S2023O1N156,RUANR5L16S2023IDS26837,SCR5A2023O1,20230227160000000,lundi 27 février 2023,1,156,AN,16,Session ordinaire 2022-2023,...,PAROLE_GENERIQUE,NORMAL,PRESIDE_DISCOURS_1_10,NaN,Mme la présidente,NaN,720908,16779.74,Le débat est clos.,point
337027,CRSANR5L16S2023O1N156,RUANR5L16S2023IDS26837,SCR5A2023O1,20230227160000000,lundi 27 février 2023,1,156,AN,16,Session ordinaire 2022-2023,...,FIN_SEAN_2_1,NORMAL,NaN,president,Mme la présidente,NaN,720908,16781.87,"Prochaine séance, ce soir, à vingt et une heur...",point
337028,CRSANR5L16S2023O1N156,RUANR5L16S2023IDS26837,SCR5A2023O1,20230227160000000,lundi 27 février 2023,1,156,AN,16,Session ordinaire 2022-2023,...,FIN_SEAN_2_4,Info Italiques,NaN,NaN,NaN,NaN,NaN,NaN,(La séance est levée à vingt heures vingt-cinq.),point


In [50]:
df_lxml

,UID,SeanceRef,SessionRef,DateSeance,DateSeanceJour,NumSeanceJour,NumSeance,TypeAssemblee,Legislature,Session,...,Ordre_seance,Code_grammaire,Code_style,Code_parole,Role_debat,Nom_orateur,Qualite_orateur,ID_orateur,stime,Texte
0,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,2,OUV_SEAN_2_1,NORMAL,NaN,president,Mme la présidente,NaN,719874.0,986.96,La séance est ouverte.
1,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,3,OUV_SEAN_2_2,Info Italiques,NaN,NaN,NaN,NaN,NaN,NaN,(La séance est ouverte à seize heures.)
2,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,7,ODJ_APPEL_DISCUSSION,NORMAL,NaN,president,Mme la présidente,NaN,719874.0,990.36,L’ordre du jour appelle la discussion du proje...
3,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,9,PAROLE_GENERIQUE,NORMAL,PAROLE_1_1,NaN,Mme la présidente,NaN,719874.0,1007.69,La parole est à Mme la ministre de la transiti...
4,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,10,PAROLE_GENERIQUE,NORMAL,PAROLE_1_2,NaN,Mme Agnès Pannier-Runacher,ministre de la transition énergétique,759832.0,1041.01,"Après la promulgation, vendredi, de la loi rel..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337036,CRSANR5L16S2023O1N156,RUANR5L16S2023IDS26837,SCR5A2023O1,20230227160000000,lundi 27 février 2023,1,156,AN,16,Session ordinaire 2022-2023,...,486,PAROLE_GENERIQUE,NORMAL,PAROLE_1_2,NaN,M. Stanislas Guerini,ministre,721498.0,16561.60,Je le dis sous le contrôle de Mme la président...
337037,CRSANR5L16S2023O1N156,RUANR5L16S2023IDS26837,SCR5A2023O1,20230227160000000,lundi 27 février 2023,1,156,AN,16,Session ordinaire 2022-2023,...,492,PAROLE_GENERIQUE,NORMAL,PRESIDE_DISCOURS_1_10,NaN,Mme la présidente,NaN,720908.0,16779.74,Le débat est clos.
337038,CRSANR5L16S2023O1N156,RUANR5L16S2023IDS26837,SCR5A2023O1,20230227160000000,lundi 27 février 2023,1,156,AN,16,Session ordinaire 2022-2023,...,495,FIN_SEAN_2_1,NORMAL,NaN,president,Mme la présidente,NaN,720908.0,16781.87,"Prochaine séance, ce soir, à vingt et une heur..."
337039,CRSANR5L16S2023O1N156,RUANR5L16S2023IDS26837,SCR5A2023O1,20230227160000000,lundi 27 février 2023,1,156,AN,16,Session ordinaire 2022-2023,...,496,FIN_SEAN_2_4,Info Italiques,NaN,NaN,NaN,NaN,NaN,NaN,(La séance est levée à vingt heures vingt-cinq.)


In [43]:
df_base["Texte"].isna().sum()

0

In [44]:
df_lxml["Texte"].isna().sum()

11

In [47]:
df_base["Nom_orateur"].value_counts()

Nom_orateur
Mme la présidente                                          81699
M. le président                                            10729
M. Éric Dupond-Moretti                                      3333
M. Gabriel Attal                                            3002
M. Sylvain Maillard                                         2903
                                                           ...  
M. Aurélien Pradié,                                            1
M. Benjamin Lucas et Mme Sandrine Rousseau                     1
Mme Constance Le Grip et plusieurs députés du groupe RE        1
Mme Laure Lavalette et Mme Caroline Parmentier                 1
M. Sylvain Maillard et M. Romain Daubié                        1
Name: count, Length: 2161, dtype: int64

In [48]:
df_lxml["Nom_orateur"].value_counts()

Nom_orateur
Mme la présidente                                          81699
M. le président                                            10729
M. Éric Dupond-Moretti                                      3333
M. Gabriel Attal                                            3002
M. Sylvain Maillard                                         2903
                                                           ...  
M. Aurélien Pradié,                                            1
M. Benjamin Lucas et Mme Sandrine Rousseau                     1
Mme Constance Le Grip et plusieurs députés du groupe RE        1
Mme Laure Lavalette et Mme Caroline Parmentier                 1
M. Sylvain Maillard et M. Romain Daubié                        1
Name: count, Length: 2161, dtype: int64